In [1]:
from itertools import islice
import pandas as pd
import zipfile
import os
import bz2
import sys

sys.path.append("../src")
from src import upload

# root = os.path.dirname(os.path.abspath(__file__))
root = '.'
os.chdir(root)
tmp_dir = os.path.join(root, "..", "tmp")

In [ ]:
name_to_file = {
    "NaturalProducts": "Enamine_REAL_natural_products_like_cxsmiles.cxsmiles.bz2",
    "Sample": "2025.02_Enamine_REAL_DB_1B.cxsmiles.bz2",
    "LeadLike": "Enamine_REAL_lead-like_cxsmiles.cxsmiles.bz2",
}

In [2]:
START = 10_000_000
END = 20_000_000
FILENAME = f"Enamine_REAL_NaturalProducts_001.csv"
FILENAME_ZIP = FILENAME + ".zip"

In [3]:
path = "/aloy/scratch/acomajuncosa/Ersilia/Enamine_libs/Enamine_REAL_natural_products_like_cxsmiles.cxsmiles.bz2"
ROWS = []

# Parse the original SMILES file
with bz2.open(path, "rt") as f:

    # Read the header
    header = f.readline().rstrip("\n").split("\t")

    # Take 'smiles' and 'id' indexes
    smiles_idx = header.index("smiles")
    id_idx = header.index("id")

    with open(os.path.join(tmp_dir, FILENAME), "w") as output_file:

        # Write header
        output_file.write("smiles\tid\n")

        # Take lines between START and END
        for line in islice(f, START, END):
            fields = line.rstrip("\n").split("\t")
            output_file.write(fields[smiles_idx])
            output_file.write("\t")
            output_file.write(fields[id_idx])
            output_file.write("\n")

# CSV to ZIP
with zipfile.ZipFile(os.path.join(tmp_dir, FILENAME_ZIP), "w", zipfile.ZIP_DEFLATED) as zf:
    zf.write(os.path.join(tmp_dir, FILENAME), arcname=FILENAME)

# Upload to Google Drive
upload(os.path.join(tmp_dir, FILENAME_ZIP), "../service.json", folder_id="1bWrCvi5FXodxQ2S88nYLecHDjk5Jer8Y")